In [10]:
import pandas as pd
import aiohttp
import asyncio
import logging
from aiohttp import ClientTimeout
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# nest_asyncio 사용
import nest_asyncio
nest_asyncio.apply()

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

KAKAO_API_KEY = ''  # KAKAO REST API 키를 여기에 입력하세요
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

# 사용자 정의 예외 클래스 (속도 제한 초과 시 발생)
class RateLimitException(Exception):
    pass

# Kakao API 호출 재시도 설정 (최대 10회 시도, 지수 증가 대기 시간)
@retry(
    stop=stop_after_attempt(10),
    wait=wait_exponential(multiplier=1, min=4, max=300),
    retry=retry_if_exception_type(RateLimitException)
)
async def get_lat_lng(session, address, semaphore):
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={address}"
    async with semaphore:  # 동시에 처리할 수 있는 요청 수를 제한
        try:
            async with session.get(url, headers=headers, timeout=ClientTimeout(total=30)) as response:
                if response.status == 200:  # 성공적으로 응답을 받은 경우
                    result = await response.json()
                    if result['documents']:
                        # 좌표(y: 위도, x: 경도)를 반환
                        return result['documents'][0]['y'], result['documents'][0]['x']
                elif response.status == 429:  # 속도 제한 초과
                    logging.warning(f"속도 제한 초과: {address}. 재시도 중...")
                    raise RateLimitException("속도 제한 초과")
                else:
                    logging.error(f"예상치 못한 상태 코드 {response.status} (주소: {address})")
        except asyncio.TimeoutError:  # 요청 시간 초과
            logging.error(f"요청 시간 초과 (주소: {address})")
        except RateLimitException:
            raise
        except Exception as e:  # 기타 오류 처리
            logging.error(f"에러 발생 (주소: {address}). 에러: {str(e)}")
    # 좌표를 찾지 못한 경우 None 반환
    return None, None

# 파일 처리 함수
async def process_file(file_path, save_interval=5000):
    data = {}  # 저장할 데이터
    count = 0
    total_processed = 0
    
    semaphore = asyncio.Semaphore(5)  # 동시에 실행할 수 있는 최대 요청 수를 5로 제한
    
    async with aiohttp.ClientSession() as session:
        with open(file_path, 'r', encoding='utf-8') as file:
            while True:
                tasks = []
                batch_data = {}
                
                # 저장 간격(save_interval)만큼 데이터를 처리
                for _ in range(save_interval):
                    line = file.readline()
                    if not line:  # 파일 끝에 도달한 경우
                        break
                    
                    # 파일에서 주소 관련 필드를 추출
                    fields = line.split('|')
                    address = f"{fields[2]} {fields[3]} {fields[4]} {fields[10]} "
                    address += f"{fields[12]}-{fields[13]}" if fields[13] != '0' else fields[12]
                    
                    zipcode = fields[16]
                    building_name = fields[-2] if fields[-2] else None
                    
                    key = (address.strip(), zipcode, building_name)
                    if key not in batch_data:
                        batch_data[key] = {'coordinates': set(), 'address': address.strip(), 'zipcode': zipcode, 'building_name': building_name}
                        # 좌표 검색 작업을 추가
                        tasks.append(asyncio.ensure_future(get_lat_lng(session, address.strip(), semaphore)))
                
                if not tasks:  # 더 이상 처리할 작업이 없으면 종료
                    break
                
                logging.info(f"{len(tasks)}개의 주소를 처리 중...")
                # 작업들을 비동기로 실행하고 결과를 수집
                lat_lng_results = await asyncio.gather(*tasks, return_exceptions=True)
                for i, result in enumerate(lat_lng_results):
                    if isinstance(result, tuple) and result[0] is not None and result[1] is not None:
                        key = list(batch_data.keys())[i]
                        batch_data[key]['coordinates'].add(result)
                    elif isinstance(result, Exception):
                        logging.error(f"에러 발생 (주소: {list(batch_data.keys())[i][0]}). 에러: {str(result)}")
                
                # 처리된 데이터를 저장할 데이터에 추가
                data.update(batch_data)
                total_processed += len(batch_data)
                
                # 데이터가 저장 간격만큼 쌓이면 CSV로 저장
                if len(data) >= save_interval:
                    save_to_csv(data, count)
                    count += len(data)
                    data = {}
                    logging.info(f"총 처리된 데이터 수: {total_processed}, 마지막 배치 크기: {len(batch_data)}")
                
                # 각 배치 처리 후 3초 대기
                await asyncio.sleep(3)
    
    if data:  # 남아있는 데이터를 저장
        save_to_csv(data, count)
        count += len(data)
    
    logging.info(f"총 저장된 데이터 수: {count}개.")
    logging.info(f"총 처리된 주소 수: {total_processed}개.")

# 데이터를 CSV 파일로 저장하는 함수
def save_to_csv(data, count):
    rows = []
    for item in data.values():
        for lat, lng in item['coordinates']:
            rows.append({
                'address': item['address'],
                'zipcode': item['zipcode'],
                'building_name': item['building_name'],
                'latitude': lat,
                'longitude': lng
            })
    
    df = pd.DataFrame(rows)
    # 저장 파일 이름 설정
    filename = f'output_with_coordinates_{count // 1000 + 1}.csv'
    df.to_csv(filename, index=False, encoding='utf-8')
    logging.info(f"{len(rows)}개 데이터를 {filename}에 저장했습니다.")

# Jupyter Notebook에서 호출할 때 사용할 메인 함수
async def main():
    file_path = '/Users/sopung/Desktop/workspace/address/rnaddrkor_seoul.txt'
    await process_file(file_path)

# Jupyter Notebook에서 실행할 때 이 줄을 사용합니다.
await main()


2024-10-02 09:40:07,002 - INFO - Processing batch of 5000 addresses...
2024-10-02 09:40:43,060 - INFO - 4996개 데이터를 output_with_coordinates_1.csv에 저장했습니다.
2024-10-02 09:40:43,061 - INFO - Total processed: 5000, Last batch size: 5000
2024-10-02 09:40:46,121 - INFO - Processing batch of 5000 addresses...
2024-10-02 09:41:21,754 - INFO - 5000개 데이터를 output_with_coordinates_6.csv에 저장했습니다.
2024-10-02 09:41:21,754 - INFO - Total processed: 10000, Last batch size: 5000
2024-10-02 09:41:24,818 - INFO - Processing batch of 5000 addresses...
2024-10-02 09:42:00,234 - INFO - 4995개 데이터를 output_with_coordinates_11.csv에 저장했습니다.
2024-10-02 09:42:00,235 - INFO - Total processed: 15000, Last batch size: 5000
2024-10-02 09:42:03,434 - INFO - Processing batch of 5000 addresses...
2024-10-02 09:42:39,245 - INFO - 4997개 데이터를 output_with_coordinates_16.csv에 저장했습니다.
2024-10-02 09:42:39,245 - INFO - Total processed: 20000, Last batch size: 5000
2024-10-02 09:42:42,304 - INFO - Processing batch of 5000 addresses

CancelledError: 

In [11]:
# 텍스트 파일에서 특정 항목 위의 텍스트 삭제하기
def remove_text_above_line(file_path, target_line_start, output_file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # 특정 라인을 찾고 그 위의 텍스트를 제거
    for i, line in enumerate(lines):
        if line.startswith(target_line_start):
            lines = lines[i:]  # 해당 라인부터 이후 내용만 남김
            break

    # 결과를 새 파일에 저장
    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.writelines(lines)

# 사용 예시
file_path = '/Users/sopung/Desktop/workspace/address/rnaddrkor_seoul.txt'  # 입력 파일 경로
output_file_path = '/Users/sopung/Desktop/workspace/address/modified_rnaddrkor_seoul.txt'  # 수정된 파일 저장 경로
target_line_start = '11410116311200800001000000'  # 특정 항목의 시작 부분

remove_text_above_line(file_path, target_line_start, output_file_path)
